### **Import packages**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable

### **Define paths**

In [0]:
silver_path = "abfss://silver@stfinancedev.dfs.core.windows.net/s_cards"
gold_path = "abfss://gold@stfinancedev.dfs.core.windows.net/dim_cards"

### **Create delta table schema**

In [0]:
spark.sql(f"""
          CREATE TABLE IF NOT EXISTS finance_cata.gold.dim_cards(
              Cardkey BIGINT GENERATED ALWAYS AS IDENTITY(START WITH 1 INCREMENT BY 1),
              CardID STRING,
              CustomerID STRING,
              AccountID STRING,
              CardNumber STRING,
              CardType STRING,
              ExpiryDate STRING,
              CVV STRING,
              CreditLimit STRING,
              CardStatus STRING,
              IssueDate TIMESTAMP,
              StartDate TIMESTAMP,
              EndDate TIMESTAMP,
              IsActive BOOLEAN
              ) USING DELTA
              LOCATION '{gold_path}'
          """)

### **Load silver data**

In [0]:
silver_df = spark.read.format("delta").load(silver_path)

### **Add SCD 2 columns**

In [0]:
silver_df = silver_df.withColumn("StartDate", current_timestamp())\
                     .withColumn("EndDate", lit(None).cast("timestamp"))\
                     .withColumn("IsActive", lit(True))

In [0]:
silver_df.display()

### **Load gold data**

In [0]:
if DeltaTable.isDeltaTable(spark, gold_path):
    dim_cards = DeltaTable.forPath(spark, gold_path)
else:
    # First-time load 
    silver_df.write.format("delta").mode("append").save(gold_path)
    dim_cards = DeltaTable.forPath(spark, gold_path)

### **Delta merge & Load data in gold layer**

In [0]:
(
    dim_cards.alias("t")
    .merge(
        silver_df.alias("s"),
        "t.CardID = s.CardID AND t.IsActive = true"
    )
    .whenMatchedUpdate(
        condition="""
        t.CustomerID <> s.CustomerID OR
        t.AccountID <> s.AccountID OR
        t.CardNumber <> s.CardNumber OR
        t.CardType <> s.CardType OR
        t.ExpiryDate <> s.ExpiryDate OR
        t.CVV <> s.CVV OR
        t.CreditLimit <> s.CreditLimit OR
        t.CardStatus <> s.CardStatus OR
        t.IssueDate <> s.IssueDate OR
        """,
        set={
            "EndDate" : current_timestamp(),
            "IsActive" :"false"
        }
    )
    .whenNotMatchedInsert(
        values={
            "CardID": "s.CardID",
            "CustomerID": "s.CustomerID",
            "AccountID": "s.AccountID",
            "CardNumber": "s.CardNumber",
            "CardType": "s.CardType",
            "ExpiryDate": "s.ExpiryDate",
            "CVV": "s.CVV",
            "CreditLimit": "s.CreditLimit",
            "CardStatus": "s.CardStatus",
            "IssueDate": "s.IssueDate",
            "StartDate": "s.StartDate",
            "EndDate": "s.EndDate",
            "IsActive": "s.IsActive"}
    )
    .execute()
)